# Use O2C_Template Customized Notebook Template

In [1]:
import pandas as pd
import numpy as np

In [2]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()

In [4]:
table_name = 'ORDER_TO_CASH_ENRICHED'

sf_df = my_session.sql("select * from {}".format(table_name))
df = sf_df.to_pandas()

In [6]:
df_train = df[df['INVOICESTATUS'] == 'Paid']
df_new = df[df['INVOICESTATUS'] != 'Paid']

In [7]:
df_train.shape, df_new.shape

((170290, 37), (3001, 37))

In [8]:
# Assuming df is your dataframe
df_train['ORDERDATE'] = pd.to_datetime(df_train['ORDERDATE'])
df_train['DELIVERYDATE'] = pd.to_datetime(df_train['DELIVERYDATE'])
df_train['INVOICEDATE'] = pd.to_datetime(df_train['INVOICEDATE'])
df_train['PAYMENTDATE'] = pd.to_datetime(df_train['PAYMENTDATE'])
df_train['INVOICEDUEDATE'] = pd.to_datetime(df_train['INVOICEDUEDATE'])

In [9]:
# Order Processing Time
df_train['ORDER_PROCESSING_TIME'] = (df_train['DELIVERYDATE'] - df_train['ORDERDATE']).dt.days

# Invoice Processing Time
df_train['INVOICE_PROCESSING_TIME'] = (df_train['PAYMENTDATE'] - df_train['INVOICEDATE']).dt.days

# Order Value per Unit
df_train['ORDER_VALUE_PER_UNIT'] = df_train['ORDERVALUE'] / df_train['ORDERQUANTITY']

In [16]:
# Delivery Delay
df_train['DELIVERY_DELAY'] = (df_train['DELIVERYDATE'] - df_train['DELIVEREDON']).dt.days

TypeError: unsupported operand type(s) for -: 'Timestamp' and 'datetime.date'

In [ ]:
# Payment Delay
df_train['PAYMENT_DELAY'] = (df_train['PAYMENTDATE'] - df_train['INVOICEDUEDATE']).dt.days

In [12]:
# Customer Lifetime Value (CLV)
clv = df_train.groupby('CUSTOMERID')['ORDERVALUE'].sum().reset_index()
clv.columns = ['CUSTOMERID', 'CUSTOMER_LIFETIME_VALUE']
df_train = df_train.merge(clv, on='CUSTOMERID', how='left')

In [14]:
# Order Frequency
order_freq = df_train.groupby('CUSTOMERID')['ORDERID'].nunique().reset_index()
order_freq.columns = ['CUSTOMERID', 'ORDER_FREQUENCY']
df_train = df_train.merge(order_freq, on='CUSTOMERID', how='left')

In [15]:
df_train.head()

,ORDERID,ORDERITEMID,PRODUCTID,PRODUCTNAME,ORDERQUANTITY,UNITPRICE,ORDERVALUE,CUSTOMERID,CUSTOMERNAME,PRODUCTCATEGORY,...,ADDRESSDETAILS,ADMINDETAILS,CREDITLIMIT,CREDITRATING,CUSTOMERTYPE,ORDER_PROCESSING_TIME,INVOICE_PROCESSING_TIME,ORDER_VALUE_PER_UNIT,CUSTOMER_LIFETIME_VALUE,ORDER_FREQUENCY
0,OR-1d383f77-5592-46ad-8e52-29d33a6502ad,OI-5759e816-1866-4073-8d3c-6ad53993c09b,PID-4b87db26-edd8-4b4a-8143-adb85ba6dbe2,Insulation Material,1630,2,3260,CID-4a14e75e-cd20-44a1-9696-d5eb1746b894,Anabrige LLP,Construction Materials,...,"5779 Michael Meadow\nSouth Connorchester, VA 9...",New Jersey,59238,fair,Construction Materials,7,2,2.0,88348548,595
1,OR-1d383f77-5592-46ad-8e52-29d33a6502ad,OI-cca9228c-f83d-4128-94d9-6b5eb944f23f,PID-89334032-db8d-4821-a9ad-5bd9a5305740,Cement,123,100,12300,CID-4a14e75e-cd20-44a1-9696-d5eb1746b894,Anabrige LLP,Construction Materials,...,"5779 Michael Meadow\nSouth Connorchester, VA 9...",New Jersey,59238,fair,Construction Materials,7,2,100.0,88348548,595
2,OR-1d383f77-5592-46ad-8e52-29d33a6502ad,OI-04e65e52-f3ab-46e6-bef4-e2c4b2c4ee3a,PID-3e2ce142-a70d-4da4-8770-2079f75f43a9,Roofing Sheets,1625,7,11375,CID-4a14e75e-cd20-44a1-9696-d5eb1746b894,Anabrige LLP,Construction Materials,...,"5779 Michael Meadow\nSouth Connorchester, VA 9...",New Jersey,59238,fair,Construction Materials,7,2,7.0,88348548,595
3,OR-343ce41a-322b-4823-8afc-52242973d163,OI-6125f927-0d99-48d5-9fa2-74b8a310af46,PID-fec6ed6f-7ac1-4655-b4a7-9766cff15b7b,Engine Oil,263,40,10520,CID-f922f515-1a7a-4640-959a-3c2f093f4d47,FULCRO,Automotive,...,"650 Peter Forks\nSouth Tonyborough, FL 20143",New York,99235,very poor,Automotive,4,13,40.0,36452740,649
4,OR-1310822f-e948-4c29-8da7-beec09196076,OI-54edae96-61a7-4666-8fe8-50d10dfd9323,PID-8b2ea2ce-e1df-4139-b179-3e7f217a9f9b,Conveyor Belt,306,1000,306000,CID-169eb6d1-27a1-4859-b7c8-5073ca1eb201,Taylor Ltd,Industrial Equipment,...,"562 Melanie Curve Apt. 149\nEast Kathleen, ID ...",South Carolina,53969,good,Industrial Equipment,7,5,1000.0,1144673360,584
